# Growth Coupling Suite - Example use case
Find strain designs that couple the succinate exchange reaction to growth in a reduced E. coli model

In [1]:
from growth_coupling_suite.gcOpt_algorithm import gcOpt
from growth_coupling_suite.strain_analysis.strain_design_analysis import StrainDesignAnalyzer
import cobra

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-05


# Load and check the config file

In [2]:
import gcOpt_config_file_single as config

## Load and prepare the model

In [3]:
# load "mid-scale" metabolic model of E. coli (https://doi.org/10.1038/srep39647)
model_name = "ECC2.json"
model = cobra.io.load_json_model("Models/" + model_name)

# prepare model
# set glucose uptake rate
model.exchanges.EX_glc__D_e.lower_bound = -10
# set oxygen uptake rate
model.exchanges.EX_o2_e.lower_bound = -20
# disable co2 uptake
model.exchanges.EX_co2_e.lower_bound = 0

## Define the target reaction

In [4]:
#  define target reaction -> succinate exchange
target_reaction = "EX_succ_e"

# preapre gcOpt config
config.exchanges_not_to_add.append(target_reaction)  

## Load heterologous reaction database model

In [5]:
hr_database_model = cobra.io.load_json_model("Models/ECC2_hr_database_dir_assessed.json")

# Run a single gcOpt instance
- The solver will still use multiple threads
- A heterologous reaction database model is built from various BIGG models, if non is manually provided 

In [6]:
# load gcOpt class
GCS = gcOpt.GCOpt(
    model,
    target_reaction,
    hr_database_model=hr_database_model,
    config=config,
    build_gcopt_problem=True
)

Remove blocked reactions...
Identify essential reactions...
Remove metabolites from biomass equation:
	 2fe2s_c  coefficient:  -2.6e-05
	 btn_c  coefficient:  -2e-06
	 cobalt2_c  coefficient:  -2.5e-05
	 mobd_c  coefficient:  -7e-06
	 udcpdp_c  coefficient:  -5.5e-05
Output directory: C:\Users\Tobi\Github\biosustain\Growth-coupling-suite\example/results_single/callback_solutions
Dualize primal problem ...
	Inner objective function minimizes EX_succ_e
Add decision variables ...
	Link deletion decision variables ...
Add inner optimality constraint (strong duality) ...
Add intervention number constraints ...
	Maximum number of interventions: 5 out of 282
	Range of addin targets: 0-4 out of 223
	Range of cofeed targets: 0-1 out of 3
	Range of source targets: 1-1 out of 6
	Range of deletion targets: 0-4 out of 50
Add outer objective function ...
	Outer objective function maximizes EX_succ_e


In [7]:
# solve gcOpt MILP problem
GCS.optimize(init_DesignAnalyzer=False)

Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 60
Set parameter Presolve to value 2
Set parameter LogFile to value "Gurobi log files/EX_succ_e_log"
Set parameter Threads to value 4
Start solving MILP ...
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 24 logical processors, using up to 4 threads

Consider calling update less frequently.

Optimize a model with 3306 rows, 4366 columns and 23940 nonzeros
Model fingerprint: 0x9e608847
Variable types: 4082 continuous, 284 integer (284 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e-02, 1e+02]
  RHS range        [3e-02, 1e+03]
Presolve removed 977 rows and 865 columns
Presolve time: 0.02s
Presolved: 2329 rows, 3501 columns, 18503 nonzeros
Variable types: 3217 continuous, 284 integer (283 binary)
> Solution dismissed. Minimum objective function constraint violated.
... saving callback solution []
Found heuri

# Analyze, save, and plot all results
- duplicate solutions are dismissed
- significant subset of interventions is determined for each design solution
- key parameters are computed for each design (coupling strength, growth rate)
- Flux spaces are plotted and saved for each design

In [8]:
sda = StrainDesignAnalyzer()

# load all strain design solution files in a directory
sda.load_strain_design_files_from_dir(
    config.output_dir,
    eval_gpr=True
    )

# summarize growth-coupling results
gc_summary = sda.growth_coupling_summary(
    results_filename="gcOpt_summary_result",
    results_dir=config.output_dir,
    determine_significant_designs=True, # duplicate solutions are disregarded,
                                        # design objects (deletions, add-ins, etc.) that do not contribute to the coupling are stripped from solution
    save_results=True,
    save_flux_space_plots=True,
    eval_gpr=True
)

Load file: gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_k4_a4_cs1_cf1.pickle
	Duplicate design solutions encountered
Reverse previous strain design...
Reverse previous strain design...
Apply parsed strain design (design_1)...
	Set medium composition
	Set design interventions
Reverse previous strain design...
Apply parsed strain design (design_2)...
	Set medium composition
	Set design interventions
Reverse previous strain design...
Apply parsed strain design (design_3)...
	Set medium composition
	Set design interventions
Reverse previous strain design...
Apply parsed strain design (design_4)...
	Set medium composition
	Set design interventions
Reverse previous strain design...
Apply parsed strain design (design_5)...
	Set medium composition
	Set design interventions
Reverse previous strain design...
Apply parsed strain design (design_6)...
	Set medium composition
	Set design interventions
	Duplicate design solutions encountered
	Duplicate design solutions encountered
	Duplicate design 

In [9]:
gc_summary

,filename,key_in_file,objective_value,number_interventions,number_genetic_interventions,interventions,interventions_excluded,genetic_interventions,unique_genetic_interventions,carbon_uptake_bounds,mutables_flux_at_maximum_growth,coupling_strength,max_growth_rate,target_flux_at_maximum_growth,score,biomass_precursor_auxotrophy
design_1,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_0,0.312843,2,3,FUM;source_glyc_e,G6PDH2r;F6PA;PAI2I__hr,"b1612,b1611,b4122",b1612;b1611;b4122,source_glyc_e:-20.0,"FUM:0.0,source_glyc_e:-20.0",0.068241,1.051551,1.096603,3.719866,
design_2,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_1,0.312843,2,3,FUM;source_glc__D_e,MALS;TPI;SUCOAS,"b1612,b1611,b4122",b1612;b1611;b4122,source_glc__D_e:-10.0,"FUM:0.0,source_glc__D_e:-10.0",0.058053,0.902541,0.94121,3.191027,
design_3,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_2,0.470296,3,5,FUM;SUCOAS;source_glc__D_e,MALS;TPI,"b1612,b1611,b4122;b0729,b0728",b1612;b1611;b4122;b0729;b0728,source_glc__D_e:-10.0,"FUM:0.0,SUCOAS:0.0,source_glc__D_e:-10.0",0.083787,0.880597,1.380516,2.021256,
design_4,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_4,1.31075,4,5,FUM;PGK;ENO;source_glyc_e,TRDR__hr,"b1612,b1611,b4122;b2926;b2779",b1612;b1611;b4122;b2926;b2779,source_glyc_e:-20.0,"FUM:0.0,PGK:0.0,ENO:0.0,source_glyc_e:-20.0",0.233151,0.757214,3.308502,1.64496,
design_5,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_8,0.470296,3,5,FUM;SUCOAS;source_glyc_e,PGK;PGM,"b1612,b1611,b4122;b0729,b0728",b1612;b1611;b4122;b0729;b0728,source_glyc_e:-20.0,"FUM:0.0,SUCOAS:0.0,source_glyc_e:-20.0",0.099992,1.033603,1.620384,2.375906,
design_6,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_9,1.31075,4,7,FUM;PGK;PGM;source_glyc_e,SUCOAS,"b1612,b1611,b4122;b2926;b3612,b4395,b0755",b1612;b1611;b4122;b2926;b3612;b4395;b0755,source_glyc_e:-20.0,"FUM:0.0,PGK:0.0,PGM:0.0,source_glyc_e:-20.0",0.233151,0.757214,3.308502,1.64496,
design_7,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_10,1.468202,5,9,FUM;PGK;PGM;SUCOAS;source_glyc_e,,"b1612,b1611,b4122;b2926;b3612,b4395,b0755;b072...",b1612;b1611;b4122;b2926;b3612;b4395;b0755;b072...,source_glyc_e:-20.0,"FUM:0.0,PGK:0.0,PGM:0.0,SUCOAS:0.0,source_glyc...",0.255802,0.745917,3.650646,1.433137,
design_8,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_13,2.148104,4,7,TPI;FUM;PGM;source_glyc_e,SUCOAS,"b3919;b1612,b1611,b4122;b3612,b4395,b0755",b3919;b1612;b1611;b4122;b3612;b4395;b0755,source_glyc_e:-20.0,"TPI:0.0,FUM:0.0,PGM:0.0,source_glyc_e:-20.0",0.320756,0.679081,4.862616,1.660694,
design_9,gcOpt_solution_dict_EX_succ_e_succinate_gc_i5_...,design_14,2.305557,5,9,TPI;FUM;PGM;SUCOAS;source_glyc_e,,"b3919;b1612,b1611,b4122;b3612,b4395,b0755;b072...",b3919;b1612;b1611;b4122;b3612;b4395;b0755;b072...,source_glyc_e:-20.0,"TPI:0.0,FUM:0.0,PGM:0.0,SUCOAS:0.0,source_glyc...",0.3385,0.669981,5.149105,1.442811,
